<a href="https://colab.research.google.com/github/jacmal/RL-Crypto-Trading-AI-BOT/blob/main/ENV_new_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO !!!
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import copy
import pandas as pd
import numpy as np
import random
from collections import deque
#from tensorboardX import SummaryWriter
from tensorflow.keras.optimizers import Adam, RMSprop
#from model import Actor_Model, Critic_Model, Shared_Model
#from utils import TradingGraph, Write_to_file, Normalizing
import matplotlib.pyplot as plt
from datetime import datetime
#from indicators import *
#from multiprocessing_env import train_multiprocessing, test_multiprocessing
import json

import requests
import io

In [ ]:
# load data sets
url_btc = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_BTCUSDT_1h.csv"
url_eth = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_ETHUSDT_1h.csv"
#url_bnb = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_BNBUSDT_1h.csv"

site_btc = requests.get(url_btc).content
#site_eth = requests.get(url_eth).content

df_btc = pd.read_csv(io.StringIO(site_btc.decode('utf-8')))
#df_eth = pd.read_csv(io.StringIO(site_eth.decode('utf-8')))

# reverse data sets
df_btc = df_btc.loc[::-1].reset_index(drop=True)
df_btc = df_btc.iloc[1:,:]

#df_eth = df_eth.loc[::-1].reset_index(drop=True)
#df_eth = df_eth.iloc[1:,:]

#
col_to_drop = ['unix', 'symbol', 'Volume USDT', 'tradecount', 'Volume BTC']

df_btc.drop(labels=col_to_drop, axis=1, inplace=True)
#df_eth.drop(labels=col_to_drop, axis=1, inplace=True)

#df_btc.set_index('date', drop=True, inplace=True)
#df_eth.set_index('date', drop=True, inplace=True)

df_btc['Open_btc'] = df_btc['open'].copy()

df_btc.head(5)

,date,open,high,low,close,Open_btc
1,2017-08-17 05-AM,4308.83,4328.69,4291.37,4315.32,4308.83
2,2017-08-17 06-AM,4315.32,4345.45,4309.37,4324.35,4315.32
3,2017-08-17 07-AM,4324.35,4349.99,4287.41,4349.99,4324.35
4,2017-08-17 08-AM,4349.99,4377.85,4333.32,4360.69,4349.99
5,2017-08-17 09-AM,4360.69,4445.78,4360.00,4444.00,4360.69


In [ ]:
# TODO !!!
################################################################################
### MAIN SETTINGS ###
#####################

# main path
FOLDER_PATH = "/content/drive/MyDrive/Projects/RL_Trading_BOT/"

# agent parameters file names
PARAMS_FILE = 'Parameters.json' # model parameters file

MODELS_FOLDER_NAME = 'RL_Models_Training'
MODEL_BEST = 'RL_Model_Best'
MODEL_NAME = 'RL_TradingAgent'
#ACTOR_NAME = '_Actor.h5'
#CRITIC_NAME = '_Critic.h5'

# environment file names
ACTUAL_DATA_FILE = 'actual_df_data.csv'       # actual crypto data
ACTUAL_DATA_NORM_FILE = 'actual_df_norm.csv'  # actual crypto data normalized
# trade history
TRADING_HISTORY_FILE = 'trade_history.csv'

################################################################################
# AGENT MODEL SETTINGS #
########################
# Reinforcement Learining - Proximal Policy Optimization (PPO) Adventage Actor Critic Model

#ACTOR_NAME = '_Actor.h5'
#CRITIC_NAME = '_Critic.h5'
LOOKBACK_WINDOW_SIZE = 10
INITIAL_MONEY_BALANCE = 1000
# Actor Critic Neural Network Hyperparameters
LEARNING_RATE = 0.00005
EPOCHS_NUMBER = 1
BATCH_SIZE = 32
# Agent Hyperparameters
GAMMA = 0.99
LAMBDA = 0.95

################################################################################
# TRADING ENVIRONMENT SETTINGS #
################################

SITE_FEES = 0.001
NORMALIZE_VALUE = 2 # 1 if NOT normalize
HOLD_CRYPTO_FACTOR_REWARD = 0.9
HOLD_MONEY_FACTOR_REWARD = 0.7
BUY_CRYPTO_FACTOR_REWARD = 0.1
#crypto_list = ['btc', 'eth', 'bnb', ...]
CRYPTO_LIST = ['btc']
simple_actions = ['buy', 'hold', 'sell']

In [ ]:
# TODO
# create program main folder
file_path = os.path.join(FOLDER_PATH, 'RL_Trading_BOT')
if not os.path.exists(file_path):
    os.makedirs(file_path)

#model_path = os.path.join(file_path, 'Model')
#if not os.path.exists(model_path):
#    os.makedirs(model_path)

In [ ]:
# TODO
def build_action_dict(crypto_list, action_list):
    '''
    TODO
    Create action dictionary.
    Args:
        crypto_list (list): List of cryptocurrency names. "['btc', 'eth', ...]"
        action_list (list): List of simple action names.  "['buy', 'sell', 'hold']"
    
    Returns:
        dict: Return dict with action number as key and action name as value.
    '''
    new_crypto_actions = ['hold_money']
    action = ''
    for crypto in crypto_list:
        for s_action in simple_actions:
            action = crypto + '_' + s_action
            new_crypto_actions.append(action)
            action = ''
    
    actions_dict = {key: val for key, val in enumerate(new_crypto_actions)}
    #actions_dict[-1] = 'episode_start'
    
    return actions_dict

ad = build_action_dict(CRYPTO_LIST, simple_actions)
print(ad)

{0: 'hold_money', 1: 'btc_buy', 2: 'btc_hold', 3: 'btc_sell'}


In [ ]:
# DEV DEV DEV
class TradingEnvironment:
    '''
    TODO
    '''
    def __init__(self, df, df_normalized, lookback_window_size=LOOKBACK_WINDOW_SIZE, initial_balance=INITIAL_MONEY_BALANCE, normalize_value=NORMALIZE_VALUE, action_dict=ad, fees=SITE_FEES):
        self.df = df                                          # dataframe with all trading data
        self.column_names = list(df_normalized.columns)[1:]   # list of columns to create a part of environment states without Date column
        self.all_df_steps = df.shape[0] - 1

        self.normalize_value = normalize_value
        self.df_normalized = self.df.iloc[:, 1:] / self.normalize_value                    # dataframe with normalized value
        
        self.lookback_window_size = lookback_window_size
        self.initial_balance = initial_balance
        self.fees = fees
        
        self.cryptos_list = CRYPTO_LIST
        
        # Action number to action label dictionary
        self.action_to_label_dict = action_dict
        # Action label to action number dictionary
        self.label_to_action_dict = {k: v for (v, k) in action_dict.items()}
        # Enviroment action space
        self.action_space = np.array(list(self.action_to_label_dict.keys()))
        
        # training history
        self.df_data_training_history = pd.DataFrame()
        
        # Market history values for the last lookback_window_size prices
        self.market_history = deque(maxlen=self.lookback_window_size)
        
        # Orders history contains the balance, net_worth, crypto_bought, crypto_sold, crypto_held values for the last lookback_window_size steps
        self.orders_history = deque(maxlen=self.lookback_window_size)

        # Orders variable dictionary
        self.orders_var_dict = None

################################################################################
# DATA #
########
    # TODO
    def add_new_data(self, data_point_df):
        '''
        Add a new data point into main DataFrame and normalized DataFrame.
        '''
        data_point_df_normalized = data_point_df / self.normalize_value

        new_df = pd.concat([self.df, data_point_df], axis=0, ignore_index=True)
        new_df.reset_index(inplace=True, drop=True)
        
        new_df_normalized = pd.concat([self.df, data_point_df_normalized], axis=0, ignore_index=True)
        new_df_normalized.reset_index(inplace=True, drop=True)


################################################################################
# ENV #
#######

    # TODO
    def init_orders_variables_dict(self):
        '''
        Initialize orders_variable_dict dictionary for orders history.
        '''
        orders_var_dict = {'_balance': self.initial_balance,
                           '_net_worth': self.initial_balance,
                           '_prev_net_worth': self.initial_balance}
        
        for c in self.cryptos_list:
            orders_var_dict['_'+ c +'_bought_price'] = 0
        
        for action in self.action_to_label_dict.values():
            orders_var_dict[action] = 0
        
        #print(orders_var_dict)
        return orders_var_dict

    # TODO
    def possible_actions(self, reset=False):
        '''
        TODO
        '''
        pass

    def build_state(self):
        pass
    
    # Get the data points for the given current_step
    # TODO
    def next_observation(self):
        '''
        TODO
        '''
        pass

    # Calculate reward
    # TODO
    def get_reward(self, reward_type):
        '''
        TODO
        '''
        pass

    # TODO
    def reset_state(self, env_steps_size=0, training=True):        
        '''
        TODO
        '''
        self.env_steps_size = env_steps_size
        
        if training==True:
            self.orders_var_dict = self.init_orders_variables_dict()

            self.episode_orders = 0 # track episode orders count
            self.prev_episode_orders = 0 # track previous episode orders count

            self.start_market_idx = random.randint(self.lookback_window_size, self.all_df_steps - env_steps_size)
            self.end_market_idx = self.start_market_idx + self.lookback_window_size

            df_market = self.df_normalized.loc[self.start_market_idx:self.end_market_idx-1, :]
            
            orders_var_dict_norm = {k: v / self.normalize_value for k,v in self.orders_var_dict.items()}
            df_orders = pd.DataFrame(orders_var_dict_norm, index=[self.start_market_idx])
            for i in range(self.start_market_idx + 1, self.end_market_idx):
                new_df = pd.DataFrame(orders_var_dict_norm, index=[i])
                df_orders = pd.concat([df_orders, new_df], axis=0)

            state_df = pd.concat([df_orders, df_market], axis=1)
            state = state_df.values
            
            return state
            
        elif training==False:
            
            state = 'TODO'

            return state


    # TODO
    def save_state(self, state_name='state_backup'):
        pass

    # TODO
    def load_state(self, state_name):
        '''
        TODO
        '''
        pass

    # TODO
    def step(self, action, training=True):
        '''
        TODO
        '''
        pass

In [ ]:
tenv = TradingEnvironment(df_btc, df_btc)

In [ ]:
s = tenv.reset_state()
print(s)

[[ 500.     500.     500.       0.       0.       0.       0.       0.
  3349.105 3360.825 3348.005 3358.55  3349.105]
 [ 500.     500.     500.       0.       0.       0.       0.       0.
  3358.55  3362.12  3334.03  3337.88  3358.55 ]
 [ 500.     500.     500.       0.       0.       0.       0.       0.
  3337.88  3352.    3333.86  3347.185 3337.88 ]
 [ 500.     500.     500.       0.       0.       0.       0.       0.
  3347.185 3349.955 3337.175 3345.    3347.185]
 [ 500.     500.     500.       0.       0.       0.       0.       0.
  3345.    3371.635 3342.5   3358.91  3345.   ]
 [ 500.     500.     500.       0.       0.       0.       0.       0.
  3358.91  3397.5   3358.9   3389.155 3358.91 ]
 [ 500.     500.     500.       0.       0.       0.       0.       0.
  3389.155 3389.73  3364.42  3378.155 3389.155]
 [ 500.     500.     500.       0.       0.       0.       0.       0.
  3378.155 3635.265 3376.505 3590.475 3378.155]
 [ 500.     500.     500.       0.       0.     

In [ ]:
s.shape

(10, 13)

In [ ]:
# Importing pandas as pd
import pandas as pd
  
# Creating the first Dataframe using dictionary
df1 = pd.DataFrame({"a":[1, 2, 3, 4],
                    "b":[5, 6, 7, 8]})
  
# Creating the Second Dataframe using dictionary
df2 = pd.DataFrame({"a":[1, 2, 3],
                    "b":[5, 6, 7], 
                    "c":[1, 5, 4]})
  
# for appending df2 at the end of df1
df1.append(df2, ignore_index = True)

,a,b,c
0,1,5,NaN
1,2,6,NaN
2,3,7,NaN
3,4,8,NaN
4,1,5,1.0
5,2,6,5.0
6,3,7,4.0
